In [3]:
from pytest import raises
from unittest.mock import Mock, call, patch

## Delay loop

In [4]:
class DatabaseError(Exception):
    pass

def retry_with_delay(func):
    delay = 1
    while True:
        try:
            return func()
        except DatabaseError:
            time.sleep(delay)
            delay *= 2
            
@patch('time.sleep')
def test_third_time_lucky(mock_sleep):
    # Simulate two failures followed by success
    mock_func = Mock(side_effect=[DatabaseError, DatabaseError, 'Yay'])

    result = retry_with_delay(mock_func)

    assert result == 'Yay'
    assert mock_func.call_count == 3
    
    # mocking sleep ensures the test runs quickly
    # while letting us check that it was called
    mock_sleep.assert_has_calls([
        call(1),
        call(2),
    ])

test_third_time_lucky()

## Mock the time

In [5]:
# This doesn't work because it's written in C.
with raises(TypeError):
    patch("datetime.date.today").start()
    # TypeError: can't set attributes of built-in/extension type 'datetime.date'

In [6]:
# But you can mock the whole module.
with patch('datetime.date') as mock_date:
    print(mock_date)

<MagicMock name='date' id='140450565516704'>


```python
# file: birthday.py

from datetime import date

def its_my_birthday():
    today = date.today()
    return today.month == 3 and today.day == 19
```

In [7]:


import birthday

@patch('birthday.date')
def test_birthday_3(mock_date):
    mock_date.today.return_value = date(2015, 3, 19)

    assert birthday.its_my_birthday()

### Freezegun

In [8]:
# Even better: use freezegun (pip install freezegun)
from freezegun import freeze_time

@freeze_time('2015-03-19')
def test_birthday_4():
    assert birthday.its_my_birthday()

## Mock the filesystem

In [9]:
from unittest.mock import mock_open

open_mock = mock_open(
    read_data='look at all my file contents'
)

with patch('__main__.open', open_mock, create=True):
    with open('myfile') as file:
        # You can read, but the file handle is not iterable.
        assert file.read() == 'look at all my file contents'

It's nice when we can read a file like this:

```python
with open('myfile') as myfile:
    for line in myfile:
        print(line)
```  

In [10]:

# We need to set up the magic methods to accomplish this.
with patch('__main__.open', mock_open(), create=True) as open_mock:
    mock_file = open_mock.return_value.__enter__.return_value
    mock_file.__iter__.return_value = ([
        'line one', 'line two'
    ])

    with open('myfile') as file:
       #assert list(file) == ['line one', 'line two']

line one
line two


## Property mocks

In [17]:
from unittest.mock import PropertyMock
from datetime import date

class Person:
    @property
    def is_birthday(self):
        today = date.today()
        return self.dob.month == today.month and self.dob.day == today.day

    @is_birthday.setter
    def is_birthday(self, value):
        pass
    
    def greet(self):
        return 'Happy birthday!' if self.is_birthday else 'Good morning!'

person = Person()

Patching the object doesn't work.

```python
with patch.object(person, 'is_birthday', return_value=True):
    assert person.greet() == 'Happy birthday!'
...
AttributeError: <__main__.Person object at 0x7f9875ea73c8> does not have the attribute 'is_birthday'
```

In [19]:
with patch.object(Person, 'is_birthday', new_callable=PropertyMock, return_value=True):
    assert person.greet() == 'Happy birthday!'